In [19]:

## Homework: Vector Search
#  In this homework, we'll experiemnt with vector with and without Elasticsearch

#  It's possible that your answers won't match exactly. If it's the case, select the closest one.

## Q1. Getting the embeddings model

#First, we will get the embeddings model `multi-qa-distilbert-cos-v1` from
#[the Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)



In [55]:

from sentence_transformers import SentenceTransformer
model_name='multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)


In [56]:

# Create the embedding for this user question:

# python
user_question = "I just discovered the course. Can I still join it?"
# 

# What's the first value of the resulting vector?

#* -0.24
#* -0.04
#* 0.07
#* 0.27

embedding_model.encode(user_question)[0]

0.078222655

In [57]:
question_embedding=embedding_model.encode(user_question)

In [58]:

## Prepare the documents
# Now we will create the embeddings for the documents.
# Load the documents with ids that we prepared in the module:
# ```python
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
#```


In [59]:
documents[0],len(documents)

({'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'id': 'c02e79ef'},
 948)

In [60]:
# We will use only a subset of the questions - the questions
# for `"machine-learning-zoomcamp"`. After filtering, you should
# have only 375 documents

## Q2. Creating the embeddings

# Now for each document, we will create an embedding for both question and answer fields.
# We want to put all of them into a single matrix `X`:

#- Create a list `embeddings`
#- Iterate over each document
#- `qa_text = f'{question} {text}'`
#- compute the embedding for `qa_text`, append to `embeddings`
#- At the end, let `X = np.array(embeddings)` (`import numpy as np`)

#What's the shape of X? (`X.shape`). Include the parantheses.
import numpy as np
embeddings=[{'qa_text':f"{doc['question']} {doc['text']}"} for doc in documents if doc['course']=="machine-learning-zoomcamp"]
X = np.array(embedding_model.encode(embeddings))

In [61]:
X.shape

(375, 768)

In [83]:
# filter_documents[0]


{'qa_text': 'How do I sign up? Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork'}

In [62]:
## Q3. Search

#We have the embeddings and the query vector. Now let's compute the
#cosine similarity between the vector from Q1 (let's call it `v`) and the matrix from Q2.

#The vectors returned from the embedding model are already
#normalized (you can check it by computing a dot product of a vector
#with itself - it should return something very close to 1.0). This means that in order
#to compute the coside similarity, it's sufficient to
#multiply the matrix `X` by the vector `v`:

#```python
v=embedding_model.encode(user_question)
scores = X.dot(v)
print(max(scores))
#```

#What's the highest score in the results?

# 65.0
# 6.5
# 0.65
# 0.065


0.6506574


In [86]:

## Vector search
# We can now compute the similarity between a query vector and all the embeddings.
# Let's use this to implement our own vector search
# ```python
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=[doc for doc in documents if doc['course']=="machine-learning-zoomcamp"], embeddings=X)
search_engine.search(v, num_results=5)

#```
# If you don't understand how the `search` function work:
# * Ask ChatGTP or any other LLM of your choice to explain the code
# * Check our pre-course workshop about implementing a search engine 
# [here](https://github.com/alexeygrigorev/build-your-own-search-engine)
# (Note: you can replace `argsort` with `argpartition` to make it a lot faster)
# 



[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [77]:
# VectorSearchEngine.__dict__

In [65]:
# v

In [66]:

## Q4. Hit-rate for our search engine
# Let's evaluate the performance of our own search engine. We will
# use the hitrate metric for evaluation.
# First, load the ground truth dataset:
# ```python
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)


In [67]:
df_ground_truth.head()

,question,course,document
0,When does the course begin?,data-engineering-zoomcamp,c02e79ef
1,How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
2,What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
3,How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
4,Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef


In [78]:

df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
df_ground_truth.shape

(1830, 3)

In [69]:

ground_truth = df_ground_truth.to_dict(orient='records')
#```


In [70]:
len(ground_truth)

1830

In [89]:
hits=0
for item in ground_truth:
    query=item['question']
    
    query_v=embedding_model.encode(query)    
    targets=search_engine.search(query_v, num_results=5)
    
    if any([obj['id']==item['document'] for obj in targets]):
        hits+=1


print(hits)
    

1720


In [93]:
print(f'hits rate : {round(hits/len(ground_truth),3)}')

hits rate : 0.94


In [94]:

#Now use the code from the module to calculate the hitrate of
#`VectorSearchEngine` with `num_results=5`.
#What did you get?

# 0.93
# 0.73
# 0.53
# 0.33


In [96]:

## Q5. Indexing with Elasticsearch
# Now let's index these documents with elasticsearch
#
#* Create the index with the same settings as in the module (but change the dimensions)
#* Index the embeddings (note: you've already computed them)
#
# After indexing, let's perform the search of the same query from Q1.
#
# What's the ID of the document with the highest score?
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},          
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
           
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [101]:
from tqdm import tqdm

In [102]:
filtered_documents=[doc for doc in documents if doc['course']=="machine-learning-zoomcamp"]

for doc in tqdm(filtered_documents):
    es_client.index(index=index_name,document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

100%|██████████| 375/375 [00:27<00:00, 13.57it/s]


In [103]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,                      
                    }
                },                
            }
        }
    }

    # response = es_client.search(index=index_name, body=search_query)
    
    # result_docs = []
    
    # for hit in response['hits']['hits']:
    #     result_docs.append(hit['_source'])
    
    return es_client.search(index=index_name, body=search_query)

In [104]:
response=elastic_search(
    query="I just discovered the course. Can I still join?",
    # course="data-engineering-zoomcamp"
)

In [105]:
response['hits']['hits'][0]

{'_index': 'course-questions',
 '_id': 'mDf1xpABgC0CfG45fCOa',
 '_score': 22.67584,
 '_source': {'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'}}

In [107]:
print(f"the ID of the document with the highest score is {response['hits']['hits'][0]['_source']['id']}")

the ID of the document with the highest score is ee58a693


In [118]:
## Q6. Hit-rate for Elasticsearch
# The search engine we used in Q4 computed the similarity between
# the query and ALL the vectors in our database. Usually this is
# not practical, as we may have a lot of data.

# Elasticsearch uses approximate techniques to make it faster.

# Let's evaluate how worse the results are when we switch from
# exact search (as in Q4) to approximate search with Elastic.

#What's hitrate for our dataset for Elastic?

#* 0.93
#* 0.73
#* 0.53
#* 0.33

hits=0
for item in ground_truth:
    query=item['question']
    
    targets=elastic_search(query)    
    # print(targets['hits']['hits'])#['_source']['id'])
    
    if any([obj['_source']['id']==item['document'] for obj in targets['hits']['hits']]):
        hits+=1


print(hits)
# type(targets)
    

1662


In [120]:
hit_rate = hits / len(ground_truth)
print(hit_rate)

0.9081967213114754


In [ ]:

# * Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2024/homework/hw3
# * It's possible that your answers won't match exactly. If it's the case, select the closest one.